In [122]:
import pandas as pd
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup
import random
import time
import pprint

In [2]:
article_df = pd.read_pickle("./naver_news_20220521.pkl")
print('article num : ',len(article_df))
article_df.head()

article num :  1209


,url,title
0,https://n.news.naver.com/mnews/article/014/000...,"오세훈측, 민주·송영길에 "" 재건축·재개발 언급 자격 있나"""
1,https://n.news.naver.com/mnews/article/003/001...,한미 정상 공동기자회견
2,https://n.news.naver.com/mnews/article/003/001...,한미 정상 공동기자회견
3,https://n.news.naver.com/mnews/article/003/001...,한미 정상 공동기자회견
4,https://n.news.naver.com/mnews/article/003/001...,한미 정상 공동기자회견


In [3]:
article_df = article_df.drop_duplicates(['url'])
print('article num : ',len(article_df))

article num :  1209


In [12]:
save_list = []
for i, row in tqdm(article_df.iterrows(), total=len(article_df)):
    title = row['title']
    news_url = row['url']
    headers = {
                "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36"
            }
    try:
        response = requests.get(news_url, headers = headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.find("div",{'class',"newsct_article"}).text
        news_dateTime = soup.find("span",{'class':"media_end_head_info_datestamp_time _ARTICLE_DATE_TIME"}).get("data-date-time")
    except:
        print("error occured")
        continue
    
    news_dict = {
        "title" : title,
        "content" : content,
        "datetime" : news_dateTime,
        "url" : news_url
    }
    save_list.append(news_dict)
#     if len(save_list) % 10 == 0:
#         print(len(save_list))
#         time.sleep(random.randint(1,3))
        
news_list_df = pd.DataFrame(save_list)
news_list_df.to_pickle("./news_data_20220521.pkl")

In [266]:
news_list_df.head()

,title,content,datetime,url
0,"오세훈측, 민주·송영길에 "" 재건축·재개발 언급 자격 있나""","\n\n""공급부족 초래한 장본인은 朴 전 시장""""서울시민 더이상 속지 않을 것"" \...",2022-05-21 23:59:02,https://n.news.naver.com/mnews/article/014/000...
1,한미 정상 공동기자회견,\n\n\n\n\n\n[서울=뉴시스] 윤석열 대통령과 조 바이든 미국 대통령이 21...,2022-05-21 23:58:11,https://n.news.naver.com/mnews/article/003/001...
2,한미 정상 공동기자회견,\n\n\n\n\n\n[서울=뉴시스] 윤석열 대통령과 조 바이든 미국 대통령이 21...,2022-05-21 23:58:11,https://n.news.naver.com/mnews/article/003/001...
3,한미 정상 공동기자회견,\n\n\n\n\n\n[서울=뉴시스] 윤석열 대통령과 조 바이든 미국 대통령이 21...,2022-05-21 23:58:11,https://n.news.naver.com/mnews/article/003/001...
4,한미 정상 공동기자회견,\n\n\n\n\n\n[서울=뉴시스] 윤석열 대통령과 조 바이든 미국 대통령이 21...,2022-05-21 23:58:11,https://n.news.naver.com/mnews/article/003/001...


In [14]:
article_df = news_list_df.drop_duplicates(['url'])
print('article num : ',len(article_df))

article num :  1209


<h3> 필요한 부분 갈무리 </h3>
<ul>
    <li>title</li>
    <li>content</li>
    <li>datetime</li>
</ul>

In [15]:
article_df.drop('url', axis=1, inplace=True)
article_df.head()

,title,content,datetime
0,"오세훈측, 민주·송영길에 "" 재건축·재개발 언급 자격 있나""","\n\n""공급부족 초래한 장본인은 朴 전 시장""""서울시민 더이상 속지 않을 것"" \...",2022-05-21 23:59:02
1,한미 정상 공동기자회견,\n\n\n\n\n\n[서울=뉴시스] 윤석열 대통령과 조 바이든 미국 대통령이 21...,2022-05-21 23:58:11
2,한미 정상 공동기자회견,\n\n\n\n\n\n[서울=뉴시스] 윤석열 대통령과 조 바이든 미국 대통령이 21...,2022-05-21 23:58:11
3,한미 정상 공동기자회견,\n\n\n\n\n\n[서울=뉴시스] 윤석열 대통령과 조 바이든 미국 대통령이 21...,2022-05-21 23:58:11
4,한미 정상 공동기자회견,\n\n\n\n\n\n[서울=뉴시스] 윤석열 대통령과 조 바이든 미국 대통령이 21...,2022-05-21 23:58:11


<h1> 전처리 </h1>

<h3> content -> document 를 위한 전처리 작업 </h3>

In [20]:
import re

In [21]:
# 전처리 함수 구성
def clean_text(text):
    text = text.replace("&lt;","<").replace("&gt;",">") # 괄호로 변경
    text = re.sub('(<([^>]+)>)', '', text) # 괄호내 문자와 괄호를 제거
    text = text.replace("&#039","").replace("&quot;","") # 의미 없는 문자 제거
    
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' 
    text = re.sub(pattern=pattern, repl='', string=text) #이메일 제거
    
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+'
    text = re.sub(pattern=pattern, repl='', string=text) # url 제거

    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  
    text = re.sub(pattern=pattern, repl='', string=text) # 자음, 모음 만 존재시 제거

    pattern = '<[^>]*>'        
    text = re.sub(pattern=pattern, repl='', string=text) # 괄호 <> 와 괄호내 문자제거 

    pattern = r'\([^)]*\)'
    text = re.sub(pattern=pattern, repl='', string=text) # 괄호 () 와 괄호내 문자제거

    pattern = r'\[[^)]*\]'
    text = re.sub(pattern=pattern, repl='', string=text) # 괄호 [] 와 괄호내 문자제거
    
    pattern = '[^\w\s.]'   
    text = re.sub(pattern=pattern, repl='', string=text) #단어, 띄어쓰기, 문자"." 이외의 특수문자 모두 제거
    text = text.strip()
    text = " ".join(text.split())

    return text 

In [22]:
test_content = article_df['content'][0]
test_content

'\n\n"공급부족 초래한 장본인은 朴 전 시장""서울시민 더이상 속지 않을 것" \n\n\n\n오세훈 국민의힘 서울시장 후보(왼쪽)와 송영길 더불어민주당 후보가 21일 서울 마포구 월드컵경기장 평화의공원에서 열린 2022 서울신문 마라톤 대회에 참석해 파이팅을 외치고 있다. 2022.5.21/뉴스1 /사진=뉴스1화상[파이낸셜뉴스] 오세훈 국민의힘 서울시장 후보측은 21일 경쟁 상대인 송영길 더불어민주당 후보와 민주당을 향해 "신속한 재건축-재개발을 언급할 자격이 있는가"라고 공세했다.   오 후보 캠프 박용찬 대변인은 이날 논평을 내고 "오늘날 부동산 시장을 이 지경으로 만든 핵심 원인은 공급 부족이며 그 공급 부족을 초래한 장본인은 바로 박원순 전 시장"이라며 이같이 말했다.   그는 "지난 19일 송영길 캠프가 블로그에 올린 카드뉴스를 접하고서 민주당의 후안무치는 역시 \'불치병\'이란 생각을 떨칠 수 없게 된다"고 말했다. 해당 카드뉴스에는 \'오세훈이 실패한 정책 송영길이 성공한다. 신속 과감한 재건축 재개발!\'이라는 문구가 담겼다.   이어 "도대체 누가 할 소리를 누가 하는 것인가"라며 "오늘날 부동산 시장을 이 지경으로 만든 핵심 원인은 공급 부족이며 그 공급 부족을 초래한 장본인은 바로 박원순 전 시장이다. 박원순 전 시장이 무려 10년간의 재임 중 주택공급의 길목에 얼마나 많은 대못을 박았는지 정녕 모른단 말인가"라고 반문했다.   그러면서 "공급에 대못을 박은 첫 번째 악수는 박원순의 \'뉴타운 해제\' 조치"라며 "박원순 전 시장 취임 이후인 2012년 \'뉴타운 해제’\'방침을 발표했으며 그로 인해 착공하지 못한 아파트가 무려 25만 가구에 달한다. 여의도 전체 가구 수가 1만여 가구이니 여의도 가구 수의 25배나 되는 아파트가 공급되지 못한 것"이라고 지적했다.   박 대변인은 "이처럼 뉴타운 공급에 대못을 박고 난 뒤 박원순 전 시장이 취한 선택은 \'도시재생\'이었다"며 "담벼락에 벽화를 그리고 정체불명의 시민단체 배만 불리는 도

In [23]:
clean_text(test_content)

'공급부족 초래한 장본인은 朴 전 시장서울시민 더이상 속지 않을 것 오세훈 국민의힘 서울시장 후보와 송영길 더불어민주당 후보가 21일 서울 마포구 월드컵경기장 평화의공원에서 열린 2022 서울신문 마라톤 대회에 참석해 파이팅을 외치고 있다. 2022.5.21뉴스1 사진뉴스1화상 오세훈 국민의힘 서울시장 후보측은 21일 경쟁 상대인 송영길 더불어민주당 후보와 민주당을 향해 신속한 재건축재개발을 언급할 자격이 있는가라고 공세했다. 오 후보 캠프 박용찬 대변인은 이날 논평을 내고 오늘날 부동산 시장을 이 지경으로 만든 핵심 원인은 공급 부족이며 그 공급 부족을 초래한 장본인은 바로 박원순 전 시장이라며 이같이 말했다. 그는 지난 19일 송영길 캠프가 블로그에 올린 카드뉴스를 접하고서 민주당의 후안무치는 역시 불치병이란 생각을 떨칠 수 없게 된다고 말했다. 해당 카드뉴스에는 오세훈이 실패한 정책 송영길이 성공한다. 신속 과감한 재건축 재개발이라는 문구가 담겼다. 이어 도대체 누가 할 소리를 누가 하는 것인가라며 오늘날 부동산 시장을 이 지경으로 만든 핵심 원인은 공급 부족이며 그 공급 부족을 초래한 장본인은 바로 박원순 전 시장이다. 박원순 전 시장이 무려 10년간의 재임 중 주택공급의 길목에 얼마나 많은 대못을 박았는지 정녕 모른단 말인가라고 반문했다. 그러면서 공급에 대못을 박은 첫 번째 악수는 박원순의 뉴타운 해제 조치라며 박원순 전 시장 취임 이후인 2012년 뉴타운 해제방침을 발표했으며 그로 인해 착공하지 못한 아파트가 무려 25만 가구에 달한다. 여의도 전체 가구 수가 1만여 가구이니 여의도 가구 수의 25배나 되는 아파트가 공급되지 못한 것이라고 지적했다. 박 대변인은 이처럼 뉴타운 공급에 대못을 박고 난 뒤 박원순 전 시장이 취한 선택은 도시재생이었다며 담벼락에 벽화를 그리고 정체불명의 시민단체 배만 불리는 도시재생 사업에 무려 4조원이 넘는 돈을 쏟아부었고 그 사이 수도 서울의 부동산 가격은 하늘 높은 줄 모르고 치솟고 있었다고 했다. 이어 박원순 시

In [24]:
content_list = []
for i, row in tqdm(article_df.iterrows(), total=len(article_df)):
    content = row['content']
    content = clean_text(content)
    row['content'] = content

In [27]:
article_df.head()

,title,content,datetime
0,"오세훈측, 민주·송영길에 "" 재건축·재개발 언급 자격 있나""",공급부족 초래한 장본인은 朴 전 시장서울시민 더이상 속지 않을 것 오세훈 국민의힘 ...,2022-05-21 23:59:02
1,한미 정상 공동기자회견,공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있...,2022-05-21 23:58:11
2,한미 정상 공동기자회견,공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있...,2022-05-21 23:58:11
3,한미 정상 공동기자회견,공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있...,2022-05-21 23:58:11
4,한미 정상 공동기자회견,공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있...,2022-05-21 23:58:11


<h3> 형태소 분석기를 이용해 content -> token_list(형태소) </h3>

In [28]:
from konlpy.tag import Komoran

In [100]:
tokens_list = []
komoran = Komoran()
for i, row in tqdm(article_df.iterrows(), total = len(article_df)):
    title = row['title']
    title = clean_text(title)
    content = row['content']
    document = title+" "+content
    tokens = [token[0] for token in komoran.pos(document)]
    tokens_list.append(tokens)

article_df['token_list'] = tokens_list

In [101]:
article_df.head()

,title,content,datetime,token_list
0,"오세훈측, 민주·송영길에 "" 재건축·재개발 언급 자격 있나""",공급부족 초래한 장본인은 朴 전 시장서울시민 더이상 속지 않을 것 오세훈 국민의힘 ...,2022-05-21 23:59:02,"[오세훈, 측, 민주, 송영길, 에, 재건축, 재, 개발, 언급, 자격, 있, 나,..."
1,한미 정상 공동기자회견,공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있...,2022-05-21 23:58:11,"[한, 미, 정상, 공동, 기자회견, 공감, 언론, 뉴시스, 가, 독자, 여러분, ..."
2,한미 정상 공동기자회견,공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있...,2022-05-21 23:58:11,"[한, 미, 정상, 공동, 기자회견, 공감, 언론, 뉴시스, 가, 독자, 여러분, ..."
3,한미 정상 공동기자회견,공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있...,2022-05-21 23:58:11,"[한, 미, 정상, 공동, 기자회견, 공감, 언론, 뉴시스, 가, 독자, 여러분, ..."
4,한미 정상 공동기자회견,공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있...,2022-05-21 23:58:11,"[한, 미, 정상, 공동, 기자회견, 공감, 언론, 뉴시스, 가, 독자, 여러분, ..."


<h3> online news 상황을 간이로 구현 <br> 시간대별로 나누기 </h3>

In [35]:
from datetime import datetime, timedelta

In [225]:
# 시작일,종료일 설정
start = "2022-05-21 00:00:00"
last = "2022-05-21 23:00:00"

# 시작일, 종료일 datetime 으로 변환
start_date = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
last_date = datetime.strptime(last, "%Y-%m-%d %H:%M:%S")

# 종료일 까지 반복
while start_date <= last_date:
    
    end_date = start_date + timedelta(hours=1) + timedelta(seconds=-1)
    
    s_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    e_date = end_date.strftime("%Y-%m-%d %H:%M:%S")
    print(s_date, e_date)
    start_date += timedelta(hours=1)

2022-05-21 00:00:00 2022-05-21 00:59:59
2022-05-21 01:00:00 2022-05-21 01:59:59
2022-05-21 02:00:00 2022-05-21 02:59:59
2022-05-21 03:00:00 2022-05-21 03:59:59
2022-05-21 04:00:00 2022-05-21 04:59:59
2022-05-21 05:00:00 2022-05-21 05:59:59
2022-05-21 06:00:00 2022-05-21 06:59:59
2022-05-21 07:00:00 2022-05-21 07:59:59
2022-05-21 08:00:00 2022-05-21 08:59:59
2022-05-21 09:00:00 2022-05-21 09:59:59
2022-05-21 10:00:00 2022-05-21 10:59:59
2022-05-21 11:00:00 2022-05-21 11:59:59
2022-05-21 12:00:00 2022-05-21 12:59:59
2022-05-21 13:00:00 2022-05-21 13:59:59
2022-05-21 14:00:00 2022-05-21 14:59:59
2022-05-21 15:00:00 2022-05-21 15:59:59
2022-05-21 16:00:00 2022-05-21 16:59:59
2022-05-21 17:00:00 2022-05-21 17:59:59
2022-05-21 18:00:00 2022-05-21 18:59:59
2022-05-21 19:00:00 2022-05-21 19:59:59
2022-05-21 20:00:00 2022-05-21 20:59:59
2022-05-21 21:00:00 2022-05-21 21:59:59
2022-05-21 22:00:00 2022-05-21 22:59:59
2022-05-21 23:00:00 2022-05-21 23:59:59


In [132]:
s_date = "2022-05-21 00:00:00"
e_date = "2022-05-21 00:59:59"

sub_df = article_df[article_df['datetime'].between(s_date,e_date)]
sub_df.index = range(len(sub_df))
sub_df

,title,content,datetime,token_list
0,"""신천지, 부정적 이미지 세탁과 교세 확장 위해 정치권 접근""",핵심요약6.1 지방선거 앞두고 신천지 정치 개입 모니터링 감시단 출범전국 신천지 피...,2022-05-21 00:27:01,"[신천지, 부정, 적, 이미지, 세탁, 과, 교세, 확장, 위하, 아, 정치, 권,..."
1,거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합),철제그릇 날아온 뒤 쳐다보는 이재명 후보 손현규 기자 인천 계양을 국회의원 보궐선거...,2022-05-21 00:24:12,"[거리, 유세, 하, 던, 이재명, 향하, 아, 철제, 그릇, 던지, ㄴ, 60, ..."
2,"이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합)",가해자 현장 체포李 큰 부상 없이 거리유세 이어 가 이재명 더불어민주당 총괄선거대책...,2022-05-21 00:11:04,"[이재명, 인천, 거리, 유세, 중, 취객, 이, 던지, ㄴ, 철제, 통, 에, 맞..."
3,"의사 출신 민주 신현영 의원, KTX 응급 환자 발생하자 달려갔다",신현영 더불어민주당 의원이 20일 KTX 열차 안에서 쓰러진 승객을 응급조치했다. ...,2022-05-21 00:10:01,"[의사, 출신, 민주, 신, 현영, 의원, KTX, 응급, 환자, 발생, 하, 자,..."
4,"한덕수 인준 '파국 피했다'…野, 지지율 추락 위기 느꼈나",尹 취임 후 10일 만에 총리 인준巨野 찬반 격론 끝에 결국 가결 당론지지율 추락에...,2022-05-21 00:04:01,"[한덕수, 인준, 파국, 피하, 았, 다, 野, 지지, 율, 추락, 위기, 느끼, ..."
5,[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘,尹 보수정권 대통령 최초 임을 위한 행진곡 제창 광주 국립묘지에서 열린 518 민중...,2022-05-21 00:02:08,"[달라지, ㄴ, 518, 기념식, 풍경, ..., 민주당, 보다, 돋보이, ㄴ, 국..."
6,"[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡'",한동훈 신드롬...장관 취임식 영상 조회수가 130만 윤석열 대통령과 조 바이든 미...,2022-05-21 00:02:04,"[윤석열, 바이든, 이재용, 3, 샷, 글로벌, 홍보, 효과, 톡톡, 한동훈, 신드..."
7,오늘 바이든 환영만찬에 여야 대표 나란히 참석,반도체공장 시찰하며 대화하는 한미 정상 취임 후 한국을 첫 방문한 조 바이든 미국 ...,2022-05-21 00:01:10,"[오늘, 바이든, 환영, 만찬, 에, 여야, 대표, 나란히, 참석, 반도체, 공장,..."
8,"[르포] 강용석의 자신감 ""지지율 10% 넘을 것"" (feat. 정유라)",당연히 완주해야군포 유세장에 100여명 모여 강용석 경기도지사 무소속 후보가 20일...,2022-05-21 00:01:08,"[강용석, 의, 자신감, 지지, 율, 10, 넘, 을, 것, 당연히, 완주, 하, ..."


In [221]:
def sub_df_return(article_df, start_date, end_date):
    sub_df = article_df[article_df['datetime'].between(start_date,end_date)]
    sub_df.index = range(len(sub_df))
    return sub_df

<h1> 군집화 </h1>

<h3> 나눠진 시간대별 기사에 군집화 </h3>

<h4> tokens -> tfidf vector </h4>

In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer

In [134]:
def build_vector(tokens_list): # tokens_list
    text = [ " ".join(tokens) for tokens in tokens_list]
    tfidf_vectorizer = TfidfVectorizer(analyzer="word", min_df = 1, ngram_range=(1,3)) #max_features = 10000
    tfidf_vectorizer.fit(text)
    vector = tfidf_vectorizer.transform(text).toarray()
    
    normalizer = Normalizer()
    vector = normalizer.fit_transform(vector)
    return vector

In [135]:
sub_df_tokens_list = []
for i, row in sub_df.iterrows():
    token_list = row['token_list']
    sub_df_tokens_list.append(token_list)

sub_df_vector = build_vector(sub_df_tokens_list)
sub_df_vector, len(sub_df_vector)

(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.00681562, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.05944597, 0.01832334, 0.01832334, ..., 0.01832334, 0.01832334,
         0.01832334]]),
 9)

In [136]:
len(sub_df_vector[0])

9956

In [222]:
def sub_vector_return(sub_df):
    sub_df_tokens_list = []
    for i, row in sub_df.iterrows():
        token_list = row['token_list']
        sub_df_tokens_list.append(token_list)

    sub_df_vector = build_vector(sub_df_tokens_list)
    return sub_df_vector

<h3> 군집화 알고리즘 1번째 AgglomerativeClustering </h3>

In [137]:
from sklearn.cluster import AgglomerativeClustering

In [174]:
def clustering_hierachy(vector, threahold=0.85):
    model = AgglomerativeClustering(linkage="average",  # vector끼리 bottom-up 방식으로 병합
                                distance_threshold = threahold, # 기준치 미만인 경우 다른 군집으로 판단
                                n_clusters=None, # 클러스터의 개수 지정X
                                affinity="cosine", # cosine similarity 방식으로 벡터 비교
                                compute_full_tree ="auto")
    model.fit_predict(vector) # 해당 벡터가 어디에 속하는지 판단
    return model.labels_ # 클러스터 반환

In [175]:
labels = clustering_hierachy(sub_df_vector)
labels

array([3, 0, 0, 5, 4, 1, 1, 1, 2], dtype=int64)

In [176]:
# 같은 라벨 끼리 묶어 보기
article_dict = {}
for i, label in enumerate(labels):
    if label not in article_dict:
        article_dict[label] = [sub_df['title'][i]]
    else:
        article_dict[label].append(sub_df['title'][i])
        
pprint.pprint(article_dict)

{0: ['거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합)', '이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합)'],
 1: ['[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘',
     "[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡'",
     '오늘 바이든 환영만찬에 여야 대표 나란히 참석'],
 2: ['[르포] 강용석의 자신감 "지지율 10% 넘을 것" (feat. 정유라)'],
 3: ['"신천지, 부정적 이미지 세탁과 교세 확장 위해 정치권 접근"'],
 4: ["한덕수 인준 '파국 피했다'…野, 지지율 추락 위기 느꼈나"],
 5: ['의사 출신 민주 신현영 의원, KTX 응급 환자 발생하자 달려갔다']}


In [235]:
def cluster_return(sub_vector, sub_df):
    labels = clustering_hierachy(sub_vector)
    # 같은 라벨 끼리 묶어 보기
    article_dict = {}
    for i, label in enumerate(labels):
        if label not in article_dict:
            article_dict[label] = [sub_df.iloc[i]]
        else:
            article_dict[label].append(sub_df.iloc[i])
    return article_dict

<h3> 군집화 알고리즘 2번째 DBSCAN </h3>

In [141]:
from sklearn.cluster import DBSCAN

In [169]:
def clustering(vector, eps=0.05, min_samples=1, metric="cosine"):
    model = DBSCAN(eps=eps,min_samples=min_samples, metric = "cosine")
    result = model.fit_predict(vector)
    return result

In [170]:
dbscan_labels = clustering(sub_df_vector)
dbscan_labels

array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=int64)

In [171]:
# 같은 라벨 끼리 묶어 보기
article_dict = {}
for i, label in enumerate(dbscan_labels):
    if label not in article_dict:
        article_dict[label] = [sub_df['title'][i]]
    else:
        article_dict[label].append(sub_df['title'][i])
        
pprint.pprint(article_dict)

{0: ['"신천지, 부정적 이미지 세탁과 교세 확장 위해 정치권 접근"'],
 1: ['거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합)'],
 2: ['이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합)'],
 3: ['의사 출신 민주 신현영 의원, KTX 응급 환자 발생하자 달려갔다'],
 4: ["한덕수 인준 '파국 피했다'…野, 지지율 추락 위기 느꼈나"],
 5: ['[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘'],
 6: ["[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡'"],
 7: ['오늘 바이든 환영만찬에 여야 대표 나란히 참석'],
 8: ['[르포] 강용석의 자신감 "지지율 10% 넘을 것" (feat. 정유라)']}


<h1> 군집 병합 </h1>

In [177]:
# 00:00:00 ~ 00:59:59 시간대 군집
article_dict

{3: ['"신천지, 부정적 이미지 세탁과 교세 확장 위해 정치권 접근"'],
 0: ['거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합)', '이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합)'],
 5: ['의사 출신 민주 신현영 의원, KTX 응급 환자 발생하자 달려갔다'],
 4: ["한덕수 인준 '파국 피했다'…野, 지지율 추락 위기 느꼈나"],
 1: ['[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘',
  "[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡'",
  '오늘 바이든 환영만찬에 여야 대표 나란히 참석'],
 2: ['[르포] 강용석의 자신감 "지지율 10% 넘을 것" (feat. 정유라)']}

In [178]:
# 01:00:00 ~ 01:59:59 시간대 군집
s_date = "2022-05-21 01:00:00"
e_date = "2022-05-21 01:59:59"

sub_df = article_df[article_df['datetime'].between(s_date,e_date)]
sub_df.index = range(len(sub_df))

sub_df_tokens_list = []
for i, row in sub_df.iterrows():
    token_list = row['token_list']
    sub_df_tokens_list.append(token_list)

sub_df_vector = build_vector(sub_df_tokens_list)
labels = clustering_hierachy(sub_df_vector)
article_dict_next = {}
for i, label in enumerate(labels):
    if label not in article_dict_next:
        article_dict_next[label] = [sub_df['title'][i]]
    else:
        article_dict_next[label].append(sub_df['title'][i])

In [179]:
article_dict_next

{0: ["尹-바이든, 최첨단 웨이퍼 서명으로 '기술동맹' 인증[바이든 방한]",
  '한국인 폭행 혐의 바이든 경호원 2명 美 송환…"마약복용 조사"'],
 1: ['이재명, 유세 중 철제그릇에 머리 맞아... 가해자 체포']}

<h3> 기존 로직 수정 필요 군집간 병합을 위한 기사 정보를 cluster에 추가 (title + more) </h3>

In [ ]:
# 같은 라벨 끼리 묶어 보기
article_dict = {}
for i, label in enumerate(labels):
    if label not in article_dict:
        article_dict[label] = [sub_df.iloc[i]]
    else:
        article_dict[label].append(sub_df.iloc[i])

In [182]:
s_date = "2022-05-21 00:00:00"
e_date = "2022-05-21 00:59:59"

sub_df = article_df[article_df['datetime'].between(s_date,e_date)]
sub_df.index = range(len(sub_df))

sub_df_tokens_list = []
for i, row in sub_df.iterrows():
    token_list = row['token_list']
    sub_df_tokens_list.append(token_list)
    
sub_df_vector = build_vector(sub_df_tokens_list)
labels = clustering_hierachy(sub_df_vector)
article_dict = {}
for i, label in enumerate(labels):
    if label not in article_dict:
        article_dict[label] = [sub_df.iloc[i]]
    else:
        article_dict[label].append(sub_df.iloc[i])

In [207]:
pprint.pprint(article_dict)

{0: [title                         거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합)
content       철제그릇 날아온 뒤 쳐다보는 이재명 후보 손현규 기자 인천 계양을 국회의원 보궐선거...
datetime                                    2022-05-21 00:24:12
token_list    [거리, 유세, 하, 던, 이재명, 향하, 아, 철제, 그릇, 던지, ㄴ, 60, ...
Name: 1, dtype: object,
     title                         이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합)
content       가해자 현장 체포李 큰 부상 없이 거리유세 이어 가 이재명 더불어민주당 총괄선거대책...
datetime                                    2022-05-21 00:11:04
token_list    [이재명, 인천, 거리, 유세, 중, 취객, 이, 던지, ㄴ, 철제, 통, 에, 맞...
Name: 2, dtype: object],
 1: [title                [주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘
content       尹 보수정권 대통령 최초 임을 위한 행진곡 제창 광주 국립묘지에서 열린 518 민중...
datetime                                    2022-05-21 00:02:08
token_list    [달라지, ㄴ, 518, 기념식, 풍경, ..., 민주당, 보다, 돋보이, ㄴ, 국...
Name: 5, dtype: object,
     title                [주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡'
content       한동훈 신드롬...장관 취임식 영상 조회수가 130만 윤석열 대통령과 조 바이든 미...
datetime   

In [208]:
pprint.pprint(article_dict_next)

{0: [title                     尹-바이든, 최첨단 웨이퍼 서명으로 '기술동맹' 인증[바이든 방한]
content       3나노 초미세공정 적용된 웨이퍼TSMC와 경쟁하는 삼성의 야심작에한미 정상 첫 만남...
datetime                                    2022-05-21 01:48:02
token_list    [尹, 바이든, 최, 첨단, 웨이퍼, 서명, 으로, 기술, 동맹, 인증, 3, 나노...
Name: 0, dtype: object,
     title                       한국인 폭행 혐의 바이든 경호원 2명 美 송환…"마약복용 조사"
content       펜스 설치된 바이든 미 대통령 숙소 김경희 특파원 한국 시민을 폭행한 혐의로 경찰에...
datetime                                    2022-05-21 01:44:09
token_list    [한국인, 폭행, 혐의, 바이든, 경호원, 2, 명, 美, 송환, 마약, 복용, 조...
Name: 1, dtype: object],
 1: [title                           이재명, 유세 중 철제그릇에 머리 맞아... 가해자 체포
content       이재명 더불어민주당 총괄선거대책위원장이 20일 인천 계양구 계산3동 일대에서 시민들...
datetime                                    2022-05-21 01:26:01
token_list    [이재명, 유세, 중, 철제, 그릇, 에, 머리, 맞, 아, ..., 가해자, 체포...
Name: 2, dtype: object]}


<h3> 이전 군집과 비교 (군집내 tokens jaccard similarity) </h3>

jaccard_similarity = |X∩Y| / |X∪Y| = 교집합 / 합집합

In [214]:
merge_dict = article_dict.copy() # 이전 군집
current_dict = article_dict_next.copy() # 현재 군집

merge_tuple = [] # 두 군집간의 병합할 군집의 숫자를 튜플로 저장
already_check = set() # 병합하기로 결정된 군집의 숫자는 다른 군집과 비교 X 하기 위해

for cluster_i_num in merge_dict:
    cluster_i = merge_dict[cluster_i_num] # 이전 군집 i
    
    cluster_i_tokens = set() # 군집 i의 token 저장
    for article in cluster_i:
        token_list = article['token_list']
        cluster_i_tokens.update(token_list)
    
    for cluster_j_num in current_dict:
        if cluster_j_num in already_check: # 이미 병합하기로 결정난 군집은 건너뛰기
            continue
        
        cluster_j = current_dict[cluster_j_num] # 현재 군집 j
        
        cluster_j_tokens = set() # 군집 j의 token 저장
        for article in cluster_j:
            token_list = article['token_list']
            cluster_j_tokens.update(token_list)
            
        intersection = set(cluster_i_tokens).intersection(cluster_j_tokens)
        union = set(cluster_i_tokens).union(cluster_j_tokens)
        if len(union) == 0:
            jaccard_similarity = 0.0
        else:
            jaccard_similarity = len(intersection) / len(union)
        if jaccard_similarity >= 0.35: # 두 token 집합 간의 유사도가 0.35 이상 일때
            merge_tuple.append((cluster_i_num, cluster_j_num)) # 병합을 위해 각 군집의 숫자를 저장
            already_check.add(cluster_j_num) # 병합 하기로 결정했기에 해당 군집은 비교에서 제외

In [215]:
merge_tuple

[(0, 1)]

In [216]:
article_dict[0],article_dict_next[1]

([title                         거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합)
  content       철제그릇 날아온 뒤 쳐다보는 이재명 후보 손현규 기자 인천 계양을 국회의원 보궐선거...
  datetime                                    2022-05-21 00:24:12
  token_list    [거리, 유세, 하, 던, 이재명, 향하, 아, 철제, 그릇, 던지, ㄴ, 60, ...
  Name: 1, dtype: object,
  title                         이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합)
  content       가해자 현장 체포李 큰 부상 없이 거리유세 이어 가 이재명 더불어민주당 총괄선거대책...
  datetime                                    2022-05-21 00:11:04
  token_list    [이재명, 인천, 거리, 유세, 중, 취객, 이, 던지, ㄴ, 철제, 통, 에, 맞...
  Name: 2, dtype: object],
 [title                           이재명, 유세 중 철제그릇에 머리 맞아... 가해자 체포
  content       이재명 더불어민주당 총괄선거대책위원장이 20일 인천 계양구 계산3동 일대에서 시민들...
  datetime                                    2022-05-21 01:26:01
  token_list    [이재명, 유세, 중, 철제, 그릇, 에, 머리, 맞, 아, ..., 가해자, 체포...
  Name: 2, dtype: object])

<h3> 병합 정보를 토대로 병합 수행 </h3>

In [217]:
print("병합 이전 : ",len(merge_dict[0]))

# 병합
for merge_info in merge_tuple:
    cluster_i_num = merge_info[0]
    cluster_j_num = merge_info[1]
    
    merge_dict[cluster_i_num].extend(current_dict[cluster_j_num])
    del current_dict[cluster_j_num]

print("병합 이후 : ", len(merge_dict[0]))

print("추가 이전 : ", len(merge_dict))
# 병합되지 않은 군집은 추가
for cluster_j_num in current_dict:
    keys = merge_dict.keys()
    keys = sorted(keys) 
    
    new_cluster_num = keys[-1]+1
    merge_dict[new_cluster_num] = current_dict[cluster_j_num]

print("추가 이후 : ", len(merge_dict))

병합 이전 :  2
병합 이후 :  3
추가 이전 :  6
추가 이후 :  7


In [228]:
item_list = [[key, len(value)] for key, value in merge_dict.items()]
item_list = sorted(item_list, key=lambda x : x[1], reverse=True)
for item in item_list:
    cluster_num = item[0]
    cluster = merge_dict[cluster_num]
    for article in cluster:
        print(article['title'])
    print("\n\n")

거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합)
이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합)
이재명, 유세 중 철제그릇에 머리 맞아... 가해자 체포



[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘
[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡'
오늘 바이든 환영만찬에 여야 대표 나란히 참석



尹-바이든, 최첨단 웨이퍼 서명으로 '기술동맹' 인증[바이든 방한]
한국인 폭행 혐의 바이든 경호원 2명 美 송환…"마약복용 조사"



"신천지, 부정적 이미지 세탁과 교세 확장 위해 정치권 접근"



의사 출신 민주 신현영 의원, KTX 응급 환자 발생하자 달려갔다



한덕수 인준 '파국 피했다'…野, 지지율 추락 위기 느꼈나



[르포] 강용석의 자신감 "지지율 10% 넘을 것" (feat. 정유라)





In [259]:
def merge(merge_dict, current_dict):
    merge_tuple = [] # 두 군집간의 병합할 군집의 숫자를 튜플로 저장
    already_check = set() # 병합하기로 결정된 군집의 숫자는 다른 군집과 비교 X 하기 위해
    for cluster_i_num in merge_dict:
        cluster_i = merge_dict[cluster_i_num] # 이전 군집 i

        cluster_i_tokens = set() # 군집 i의 token 저장
        for article in cluster_i:
            token_list = article['token_list']
            cluster_i_tokens.update(token_list)

        for cluster_j_num in current_dict:
            if cluster_j_num in already_check: # 이미 병합하기로 결정난 군집은 건너뛰기
                continue

            cluster_j = current_dict[cluster_j_num] # 현재 군집 j

            cluster_j_tokens = set() # 군집 j의 token 저장
            for article in cluster_j:
                token_list = article['token_list']
                cluster_j_tokens.update(token_list)

            intersection = set(cluster_i_tokens).intersection(cluster_j_tokens)
            union = set(cluster_i_tokens).union(cluster_j_tokens)
            if len(union) == 0:
                jaccard_similarity = 0.0
            else:
                jaccard_similarity = len(intersection) / len(union)
            if jaccard_similarity >= 0.35: # 두 token 집합 간의 유사도가 0.35 이상 일때
                merge_tuple.append((cluster_i_num, cluster_j_num)) # 병합을 위해 각 군집의 숫자를 저장
                already_check.add(cluster_j_num) # 병합 하기로 결정했기에 해당 군집은 비교에서 제외
                
    # 병합
    for merge_info in merge_tuple:
        cluster_i_num = merge_info[0]
        cluster_j_num = merge_info[1]

        merge_dict[cluster_i_num].extend(current_dict[cluster_j_num])
        del current_dict[cluster_j_num]

    # 병합되지 않은 군집은 추가
    for cluster_j_num in current_dict:
        keys = merge_dict.keys()
        keys = sorted(keys) 

        new_cluster_num = keys[-1]+1
        merge_dict[new_cluster_num] = current_dict[cluster_j_num]

    return merge_dict

<h4> 
시간대 별로 병합 수행 <br><br>
00:00:00~ 00:59:59 군집화 -> 이전이 없기 때문에 병합 x<br><br>
01:00:00~ 01:59:59 군집화 -> 이전 군집과 병합<br><br>
02:00:00~ 02:59:59 군집화 -> 이전 군집과 병합<br><br>
    ...<br><br>
</h4>

In [260]:
# 시작일,종료일 설정
start = "2022-05-21 00:00:00"
last = "2022-05-21 23:00:00"

# 시작일, 종료일 datetime 으로 변환
start_date = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
last_date = datetime.strptime(last, "%Y-%m-%d %H:%M:%S")

merge_dict = {}
current_dict = {}

# 종료일 까지 반복
while start_date <= last_date:
    
    end_date = start_date + timedelta(hours=1) + timedelta(seconds=-1)
    
    s_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    e_date = end_date.strftime("%Y-%m-%d %H:%M:%S")
    print(s_date, e_date)
    
    sub_df = sub_df_return(article_df, s_date, e_date)
    sub_vector = sub_vector_return(sub_df)
    article_dict = cluster_return(sub_vector, sub_df)
    
    if len(merge_dict) == 0:
        merge_dict = article_dict.copy()
    else:
        current_dict = article_dict.copy()
        merge_dict = merge(merge_dict, current_dict).copy()
    
    print("**************",s_date,"~",e_date,"군집화 결과********************")
    item_list = [[key, len(value)] for key, value in merge_dict.items()]
    item_list = sorted(item_list, key=lambda x : x[1], reverse=True)
    for item in item_list[:5]:
        cluster_num = item[0]
        cluster = merge_dict[cluster_num]
        for article in cluster:
            print(article['title'], article['datetime'])
        print("\n")
    start_date += timedelta(hours=1)

2022-05-21 00:00:00 2022-05-21 00:59:59
************** 2022-05-21 00:00:00 ~ 2022-05-21 00:59:59 군집화 결과********************
[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘 2022-05-21 00:02:08
[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡' 2022-05-21 00:02:04
오늘 바이든 환영만찬에 여야 대표 나란히 참석 2022-05-21 00:01:10


거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합) 2022-05-21 00:24:12
이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합) 2022-05-21 00:11:04


"신천지, 부정적 이미지 세탁과 교세 확장 위해 정치권 접근" 2022-05-21 00:27:01


의사 출신 민주 신현영 의원, KTX 응급 환자 발생하자 달려갔다 2022-05-21 00:10:01


한덕수 인준 '파국 피했다'…野, 지지율 추락 위기 느꼈나 2022-05-21 00:04:01


2022-05-21 01:00:00 2022-05-21 01:59:59
************** 2022-05-21 01:00:00 ~ 2022-05-21 01:59:59 군집화 결과********************
거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합) 2022-05-21 00:24:12
이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합) 2022-05-21 00:11:04
이재명, 유세 중 철제그릇에 머리 맞아... 가해자 체포 2022-05-21 01:26:01


[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘 2022-05-21 00:02:08
[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡' 2022-05-21 00:02:04
오늘 바이든 환영

************** 2022-05-21 10:00:00 ~ 2022-05-21 10:59:59 군집화 결과********************
대회 참가자들과 기념촬영하는 송영길·오세훈 후보 2022-05-21 09:52:49
마라톤 참가자들 향해 표심 호소하는 송영길·오세훈 후보 2022-05-21 09:52:28
송영길·오세훈 후보, 마라톤 참가자들과 하이파이브 2022-05-21 09:51:55
송영길·오세훈 후보, 참가자들 화이팅! 2022-05-21 09:51:21
마라톤 참가자들과 하이파이브 하는 송영길·오세훈 후보 2022-05-21 09:50:39
마라톤 행사 참석한 송영길 후보와 오세훈 후보 2022-05-21 09:50:14
대회 참가자들에게 인사하는 송영길 후보 2022-05-21 09:48:41
서울신문 마라톤 대회 참석한 송영길·오세훈 후보 2022-05-21 09:48:35
대화하는 송영길 후보와 오세훈 후보 2022-05-21 09:48:13
대화하는 송영길 후보와 오세훈 후보 2022-05-21 09:48:12
대화하는 송영길 후보와 오세훈 후보 2022-05-21 09:48:11
대회 참가자들과 인사 나누는 송영길 후보 2022-05-21 10:59:26
마라톤 대회 참석한 오세훈 후보 2022-05-21 10:59:21
정청래 의원, 송영길 후보 응원해요 2022-05-21 10:58:56
시민과 인사하는 송영길 후보 2022-05-21 10:48:10
국민의힘 후보들과 인사하는 송영길 후보 2022-05-21 10:45:16
손잡은 만남 송영길 후보와 오세훈 후보 2022-05-21 10:41:10
마라톤 대회에서 만난 송영길 후보와 오세훈 후보 2022-05-21 10:40:15
마라톤 대회에서 만난 송영길 후보와 오세훈 후보 2022-05-21 10:39:13
마라톤 대회에서 만난 송영길 후보와 오세훈 후보 2022-05-21 10:39:12
마라톤 대회에서 만난 송영길 후보와 오세훈 후보 2022-05-21 10:

************** 2022-05-21 12:00:00 ~ 2022-05-21 12:59:59 군집화 결과********************
尹 대통령, 한덕수 국무총리 임명… 지명 48일만 2022-05-21 10:56:02
尹 대통령, 한덕수 국무총리 임명 2022-05-21 10:44:01
윤석열 대통령, 한덕수 국무총리 임명..."같이 열심히 일하자" 2022-05-21 10:39:01
尹대통령, 한덕수 국무총리 임명... 1기 내각 본격 출범 2022-05-21 10:34:01
윤 대통령, 한덕수 신임 국무총리에 임명장 수여[경향포토] 2022-05-21 10:32:02
윤 대통령, 한덕수 국무총리 임명… 후보자 지목 48일만(상보) 2022-05-21 10:23:04
尹대통령, 한덕수 국무총리 임명…지명 48일만 2022-05-21 10:20:10
尹대통령, 한덕수 총리 임명 재가…지명 후 48일만 2022-05-21 10:17:07
尹대통령, 한덕수 국무총리 임명…지명 후 48일 만 2022-05-21 10:12:22
[속보] 尹 대통령, 한덕수 국무총리 임명 재가 2022-05-21 10:10:11
[속보] 윤 대통령, 한덕수 국무총리 임명 2022-05-21 10:10:01
[속보] 尹 대통령, 한덕수 국무총리 임명 재가 2022-05-21 10:05:52
[속보] 윤석열 대통령, 한덕수 국무총리 임명 재가 2022-05-21 10:05:02
윤 대통령, 한덕수 총리 임명…“열심히 일해달라” 2022-05-21 10:04:50
윤 대통령, 한덕수 국무총리 임명안 재가…임명장 수여 2022-05-21 10:04:49
[속보] 尹대통령, 한덕수 국무총리 임명 2022-05-21 10:04:13
尹 대통령, 한덕수 국무총리 임명 재가 2022-05-21 10:04:02
[속보] 尹대통령, 한덕수 국무총리 임명 2022-05-21 10:03:41
尹대통령, 한덕수 국무총리 임명…지명 48일 만 2022-05-21 10:03:17
윤 대통령, 

************** 2022-05-21 14:00:00 ~ 2022-05-21 14:59:59 군집화 결과********************
지지호소에 나선 나동연 국힘 양산시장 후보 2022-05-21 08:36:15
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
이창양 장관과 회담 하는 지나 레이몬도 장관 2022-05-21 10:46:25
지나 레이몬도 미국 상무부 장관과 회담 하는 이창양 장관 2022-05-21 10:46:25
회담 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
환담장 향하는 윤석열 대통령-한덕수 신임 국무총리 2022-05-21 10:41:59
'선거공보 및 투표 안내문 발송 작업' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
선거 공보물 발송에 분주 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'분주한 손놀림' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
한덕수 국무총리에게 악수 건네는 윤 대통령 2022-05-21 10:23:24
임명장 수여식 기념촬영하는 윤석열 대통령과 한덕수 신임 국무총리 2022-05-21 10:23:13
시민들에게 지지호소하는 김두겸 울산시장 후보 2022-05-21 12:48:37
'한표 부탁합

************** 2022-05-21 16:00:00 ~ 2022-05-21 16:59:59 군집화 결과********************
윤석열-바이든, 잠시 뒤 용산 대통령실에서 90분 정상회담 2022-05-21 12:55:01
[포토] 용산 대통령실 청사 도착한 바이든 미 대통령 2022-05-21 13:50:01
바이든, 용산 대통령실 도착…윤 대통령과 정상회담 돌입 2022-05-21 13:49:04
[속보] 바이든, 서울 용산 대통령실 도착…尹 대통령, 정문 밖에서 영접 2022-05-21 13:45:01
윤석열 대통령, 용산 청사 정문서 바이든 영접…정상회담 후 공동기자회견 2022-05-21 13:40:01
바이든 미국 대통령 영접하는 윤석열 대통령 2022-05-21 13:31:11
바이든 미국 대통령 영접하는 윤석열 대통령 2022-05-21 13:30:10
[속보] 바이든, 용산 대통령실 청사 도착...尹대통령, 정문 입구서 영접 2022-05-21 13:26:01
[속보] 바이든, 용산 대통령실 도착…尹, 정문서 직접 영접 2022-05-21 13:26:01
[영상] 바이든, 용산 대통령실 도착…윤 대통령, 정문서 직접 영접 2022-05-21 13:25:37
[속보] 바이든, 용산 대통령실 청사 도착 2022-05-21 13:25:19
[속보] 바이든, 용산 대통령실 청사 도착 2022-05-21 13:25:04
美 바이든 대통령 방한 둘째 날 현충원 참배…곧 정상회담 2022-05-21 13:13:43
尹, 용산청사 정문서 바이든 직접 영접…정상회담 시작 2022-05-21 13:12:01
尹·바이든, 90분 정상회담 시작…소인수→5분간 환담→확대 2022-05-21 14:48:57
용산 청사 온 바이든, 방명록에 "환대와 동맹에 감사"[바이든 방한] 2022-05-21 14:45:02
尹대통령, 용산청사 ‘레드카펫’서 바이든 맞이…정상회담 돌입(종합) 2022-05-21 14:42:01
확대정상회담에 한미 주요 인사 22

...

************** 2022-05-21 19:00:00 ~ 2022-05-21 19:59:59 군집화 결과********************
지지호소에 나선 나동연 국힘 양산시장 후보 2022-05-21 08:36:15
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
이창양 장관과 회담 하는 지나 레이몬도 장관 2022-05-21 10:46:25
지나 레이몬도 미국 상무부 장관과 회담 하는 이창양 장관 2022-05-21 10:46:25
회담 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
환담장 향하는 윤석열 대통령-한덕수 신임 국무총리 2022-05-21 10:41:59
'선거공보 및 투표 안내문 발송 작업' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
선거 공보물 발송에 분주 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'분주한 손놀림' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
한덕수 국무총리에게 악수 건네는 윤 대통령 2022-05-21 10:23:24
임명장 수여식 기념촬영하는 윤석열 대통령과 한덕수 신임 국무총리 2022-05-21 10:23:13
시민들에게 지지호소하는 김두겸 울산시장 후보 2022-05-21 12:48:37
'한표 부탁합

************** 2022-05-21 20:00:00 ~ 2022-05-21 20:59:59 군집화 결과********************
지지호소에 나선 나동연 국힘 양산시장 후보 2022-05-21 08:36:15
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
이창양 장관과 회담 하는 지나 레이몬도 장관 2022-05-21 10:46:25
지나 레이몬도 미국 상무부 장관과 회담 하는 이창양 장관 2022-05-21 10:46:25
회담 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
환담장 향하는 윤석열 대통령-한덕수 신임 국무총리 2022-05-21 10:41:59
'선거공보 및 투표 안내문 발송 작업' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
선거 공보물 발송에 분주 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'분주한 손놀림' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
한덕수 국무총리에게 악수 건네는 윤 대통령 2022-05-21 10:23:24
임명장 수여식 기념촬영하는 윤석열 대통령과 한덕수 신임 국무총리 2022-05-21 10:23:13
시민들에게 지지호소하는 김두겸 울산시장 후보 2022-05-21 12:48:37
'한표 부탁합

************** 2022-05-21 21:00:00 ~ 2022-05-21 21:59:59 군집화 결과********************
지지호소에 나선 나동연 국힘 양산시장 후보 2022-05-21 08:36:15
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
이창양 장관과 회담 하는 지나 레이몬도 장관 2022-05-21 10:46:25
지나 레이몬도 미국 상무부 장관과 회담 하는 이창양 장관 2022-05-21 10:46:25
회담 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
환담장 향하는 윤석열 대통령-한덕수 신임 국무총리 2022-05-21 10:41:59
'선거공보 및 투표 안내문 발송 작업' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
선거 공보물 발송에 분주 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'분주한 손놀림' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
한덕수 국무총리에게 악수 건네는 윤 대통령 2022-05-21 10:23:24
임명장 수여식 기념촬영하는 윤석열 대통령과 한덕수 신임 국무총리 2022-05-21 10:23:13
시민들에게 지지호소하는 김두겸 울산시장 후보 2022-05-21 12:48:37
'한표 부탁합

************** 2022-05-21 22:00:00 ~ 2022-05-21 22:59:59 군집화 결과********************
지지호소에 나선 나동연 국힘 양산시장 후보 2022-05-21 08:36:15
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
이창양 장관과 회담 하는 지나 레이몬도 장관 2022-05-21 10:46:25
지나 레이몬도 미국 상무부 장관과 회담 하는 이창양 장관 2022-05-21 10:46:25
회담 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
인사 하는 이창양 장관-지나 레이몬도 장관 2022-05-21 10:46:25
환담장 향하는 윤석열 대통령-한덕수 신임 국무총리 2022-05-21 10:41:59
'선거공보 및 투표 안내문 발송 작업' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
선거 공보물 발송에 분주 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송' 2022-05-21 10:24:50
'후보들의 공약을 확인할 수 있는 선거공보 발송 준비에 분주' 2022-05-21 10:24:50
'분주한 손놀림' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'번호 순서대로 하나씩' 2022-05-21 10:24:50
'제8회 전국동시지방선거 선거공보 및 투표안내문 발송 준비' 2022-05-21 10:24:50
한덕수 국무총리에게 악수 건네는 윤 대통령 2022-05-21 10:23:24
임명장 수여식 기념촬영하는 윤석열 대통령과 한덕수 신임 국무총리 2022-05-21 10:23:13
시민들에게 지지호소하는 김두겸 울산시장 후보 2022-05-21 12:48:37
'한표 부탁합

************** 2022-05-21 23:00:00 ~ 2022-05-21 23:59:59 군집화 결과********************
박형준 후보, 시민들과 만남 통해 지지 호소 2022-05-21 17:56:52
공동기자회견 마친 한미 정상 2022-05-21 17:51:31
공동기자회견 마친 한미 정상 2022-05-21 17:51:31
답변하는 윤석열 대통령 2022-05-21 17:51:31
한미 정상회담 관련 브리핑하는 김성한 안보실장 2022-05-21 17:51:01
한미 정상회담 관련 브리핑하는 김성한 안보실장 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 2022-05-21 17:51:01
한미 정상회담 관련 브리핑하는 김성한 안보실장 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 2022-05-21 17:51:01
변성완 부산시장 후보 지지 호소 2022-05-21 17:37:06
공동기자회견하는 한미 정상 2022-05-21 17:36:55
공동기자회견하는 한미 정상 2022-05-21 17:36:55
두 정상 발언 경청하는 추경호 경제부총리 2022-05-21 17:21:40
공동기자회견하는 한미 정상 2022-05-21 17:15:43
한미 정상 공동기자회견 2022-05-21 17:15:43
공동기자회견하는 한미 정상 2022-05-21 17:15:43
공동기자회견하는 한미 정상 2022-05-21 17:15:43
한미 정상 공동기자회견 2022-05-21 17:15:27
공동기자회견하는 한미 정상 2022-05-21 17:15:27
한미 정상 공동기자회견 2022-05-21 17:15:27
한미 정상 공동기자회견 2022-05-21 17:15:27
공동기자회견 참석하는 한미 정상 2022-05-21 17:12:01
공동기자회견 참석하는 한미 정상 2022-05-21 

<h4> 
    군집의 크기가 무한정 커지기 시작...! <br><br>
    시간이 오래된 기사를 군집내 제거 필요! <br><br>
    임의 기준 (4시간 이상 되면 제거)
</h4>

In [240]:
# 기사 시간 차이 구하기 예제
start = "2022-05-21 00:00:00"
start_date = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")

a_time = article_df['datetime'][0]
a_date = datetime.strptime(a_time, "%Y-%m-%d %H:%M:%S")

diff = a_date - start_date
print(diff.seconds)
if diff.seconds > 100:
    print("true")

86342
true


In [261]:
# 시작일,종료일 설정
start = "2022-05-21 00:00:00"
last = "2022-05-21 23:00:00"

# 시작일, 종료일 datetime 으로 변환
start_date = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
last_date = datetime.strptime(last, "%Y-%m-%d %H:%M:%S")

merge_dict = {}
current_dict = {}

# 종료일 까지 반복
while start_date <= last_date:
    
    end_date = start_date + timedelta(hours=1) + timedelta(seconds=-1)
    
    s_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    e_date = end_date.strftime("%Y-%m-%d %H:%M:%S")
    print(s_date, e_date)
    
    sub_df = sub_df_return(article_df, s_date, e_date)
    sub_vector = sub_vector_return(sub_df)
    article_dict = dict(cluster_return(sub_vector, sub_df))
    if len(merge_dict) == 0:
        merge_dict = article_dict.copy()
    else:
        current_dict = article_dict.copy()
        merge_dict = merge(merge_dict, current_dict).copy()
    
    print("**************",s_date,"~",e_date,"군집화 결과********************")
    item_list = [[key, len(value)] for key, value in merge_dict.items()]
    item_list = sorted(item_list, key=lambda x : x[1], reverse=True)
    for item in item_list[:5]:
        cluster_num = item[0]
        cluster = merge_dict[cluster_num]
        for article in cluster:
            print(article['title'],"\t",article['datetime'])
        print("\n")
        
    # 시간이 오래된 값은 제거 필요
    remove_cluster_list = []
    for cluster_num in merge_dict:
        cluster = merge_dict[cluster_num]
        remove_list = []
        # 시간 차이를 구한뒤, 리스트의 index를 저장
        for i, article in enumerate(cluster):
            article_time = article['datetime']
            article_time = datetime.strptime(article_time, "%Y-%m-%d %H:%M:%S")
            
            diff = article_time - start_date
            if diff.seconds > 14400: # 4시간 이상 차이날 경우 제거 필요
                remove_list.append(i)
                
        # 지워야할 index를 pop 해준다
        remove_list.reverse() # 뒤에서 부터 지워야 하기 때문에 reverse 함수
        for i in remove_list:
            cluster.pop(i)
            
        # 지우고 보니 cluster에 남아있는게 없다면 해당 클러스터 제거
        if len(cluster) == 0:
            remove_cluster_list.append(cluster_num)
            
    # 지우고 보니 cluster에 남아있는게 없다면 해당 클러스터 제거
    for cluster_num in remove_cluster_list:
        del merge_dict[cluster_num]
        
    start_date += timedelta(hours=1)

2022-05-21 00:00:00 2022-05-21 00:59:59
************** 2022-05-21 00:00:00 ~ 2022-05-21 00:59:59 군집화 결과********************
[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘 	 2022-05-21 00:02:08
[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡' 	 2022-05-21 00:02:04
오늘 바이든 환영만찬에 여야 대표 나란히 참석 	 2022-05-21 00:01:10


거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합) 	 2022-05-21 00:24:12
이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합) 	 2022-05-21 00:11:04


"신천지, 부정적 이미지 세탁과 교세 확장 위해 정치권 접근" 	 2022-05-21 00:27:01


의사 출신 민주 신현영 의원, KTX 응급 환자 발생하자 달려갔다 	 2022-05-21 00:10:01


한덕수 인준 '파국 피했다'…野, 지지율 추락 위기 느꼈나 	 2022-05-21 00:04:01


2022-05-21 01:00:00 2022-05-21 01:59:59
************** 2022-05-21 01:00:00 ~ 2022-05-21 01:59:59 군집화 결과********************
거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합) 	 2022-05-21 00:24:12
이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합) 	 2022-05-21 00:11:04
이재명, 유세 중 철제그릇에 머리 맞아... 가해자 체포 	 2022-05-21 01:26:01


[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘 	 2022-05-21 00:02:08
[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡' 	 202

************** 2022-05-21 11:00:00 ~ 2022-05-21 11:59:59 군집화 결과********************
尹 대통령, 한덕수 국무총리 임명… 지명 48일만 	 2022-05-21 10:56:02
尹 대통령, 한덕수 국무총리 임명 	 2022-05-21 10:44:01
윤석열 대통령, 한덕수 국무총리 임명..."같이 열심히 일하자" 	 2022-05-21 10:39:01
尹대통령, 한덕수 국무총리 임명... 1기 내각 본격 출범 	 2022-05-21 10:34:01
윤 대통령, 한덕수 신임 국무총리에 임명장 수여[경향포토] 	 2022-05-21 10:32:02
윤 대통령, 한덕수 국무총리 임명… 후보자 지목 48일만(상보) 	 2022-05-21 10:23:04
尹대통령, 한덕수 국무총리 임명…지명 48일만 	 2022-05-21 10:20:10
尹대통령, 한덕수 총리 임명 재가…지명 후 48일만 	 2022-05-21 10:17:07
尹대통령, 한덕수 국무총리 임명…지명 후 48일 만 	 2022-05-21 10:12:22
[속보] 尹 대통령, 한덕수 국무총리 임명 재가 	 2022-05-21 10:10:11
[속보] 윤 대통령, 한덕수 국무총리 임명 	 2022-05-21 10:10:01
[속보] 尹 대통령, 한덕수 국무총리 임명 재가 	 2022-05-21 10:05:52
[속보] 윤석열 대통령, 한덕수 국무총리 임명 재가 	 2022-05-21 10:05:02
윤 대통령, 한덕수 총리 임명…“열심히 일해달라” 	 2022-05-21 10:04:50
윤 대통령, 한덕수 국무총리 임명안 재가…임명장 수여 	 2022-05-21 10:04:49
[속보] 尹대통령, 한덕수 국무총리 임명 	 2022-05-21 10:04:13
尹 대통령, 한덕수 국무총리 임명 재가 	 2022-05-21 10:04:02
[속보] 尹대통령, 한덕수 국무총리 임명 	 2022-05-21 10:03:41
尹대통령, 한덕수 국무총리 임명…

바이든, 용산 대통령실 도착…윤 대통령과 정상회담 돌입 	 2022-05-21 13:49:04
[속보] 바이든, 서울 용산 대통령실 도착…尹 대통령, 정문 밖에서 영접 	 2022-05-21 13:45:01
윤석열 대통령, 용산 청사 정문서 바이든 영접…정상회담 후 공동기자회견 	 2022-05-21 13:40:01
바이든 미국 대통령 영접하는 윤석열 대통령 	 2022-05-21 13:31:11
바이든 미국 대통령 영접하는 윤석열 대통령 	 2022-05-21 13:30:10
[속보] 바이든, 용산 대통령실 청사 도착...尹대통령, 정문 입구서 영접 	 2022-05-21 13:26:01
[속보] 바이든, 용산 대통령실 도착…尹, 정문서 직접 영접 	 2022-05-21 13:26:01
[영상] 바이든, 용산 대통령실 도착…윤 대통령, 정문서 직접 영접 	 2022-05-21 13:25:37
[속보] 바이든, 용산 대통령실 청사 도착 	 2022-05-21 13:25:19
[속보] 바이든, 용산 대통령실 청사 도착 	 2022-05-21 13:25:04
美 바이든 대통령 방한 둘째 날 현충원 참배…곧 정상회담 	 2022-05-21 13:13:43
尹, 용산청사 정문서 바이든 직접 영접…정상회담 시작 	 2022-05-21 13:12:01
尹·바이든, 90분 정상회담 시작…소인수→5분간 환담→확대 	 2022-05-21 14:48:57
용산 청사 온 바이든, 방명록에 "환대와 동맹에 감사"[바이든 방한] 	 2022-05-21 14:45:02
尹대통령, 용산청사 ‘레드카펫’서 바이든 맞이…정상회담 돌입(종합) 	 2022-05-21 14:42:01
확대정상회담에 한미 주요 인사 22명 참석...尹, 바이든과 공동선언문 발표 예정 	 2022-05-21 14:31:01
바이든. 현충원 참배…“나라 위해 모든 것 바친 영웅에 경의” 	 2022-05-21 14:30:01
바이든, 현충원 참배…"나라에 모든 것 바친 영웅들에 경의" 	 2022-05-21 14:2

************** 2022-05-21 16:00:00 ~ 2022-05-21 16:59:59 군집화 결과********************
회담 참석자와 인사하는 윤석열 대통령 	 2022-05-21 15:58:19
[속보]바이든 "한미동맹, 北 위협 억제에 중요..오늘날 세계 안전유지에 필수" 	 2022-05-21 15:58:03
[속보] 尹대통령 "한미, 반도체·배터리 등 전략산업 분야 협력" 	 2022-05-21 15:56:13
[속보] 尹대통령 "한미동맹, 경제안보시대 맞춰 발전·진화해야" 	 2022-05-21 15:56:13
한미 정상회담 	 2022-05-21 15:55:11
한미 정상회담 	 2022-05-21 15:55:10
한미 정상회담 	 2022-05-21 15:55:09
[속보] 바이든 “한미동맹, 세계의 안전 유지하는 데 필수적” 	 2022-05-21 15:54:01
[속보영상] 용산 대통령실 ‘첫 외빈’ 바이든 “환대와 동맹에 감사” 	 2022-05-21 15:53:15
[속보] 한미 확대정상회담 종료…곧 기자회견 	 2022-05-21 15:47:10
회담 참석자와 인사하는 윤석열 대통령 	 2022-05-21 15:41:13
회담 참석자와 인사하는 윤석열 대통령 	 2022-05-21 15:41:12
대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:40:20
대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:40:19
발언하는 윤석열 대통령 	 2022-05-21 15:39:12
발언하는 바이든 대통령 	 2022-05-21 15:38:15
발언하는 바이든 대통령 	 2022-05-21 15:38:13
용산 대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:22:13
용산 대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:22:13
용산 대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:22:12
용산 대통령실 청사에서 열린 한미 

************** 2022-05-21 18:00:00 ~ 2022-05-21 18:59:59 군집화 결과********************
박형준 후보, 시민들과 만남 통해 지지 호소 	 2022-05-21 17:56:52
공동기자회견 마친 한미 정상 	 2022-05-21 17:51:31
공동기자회견 마친 한미 정상 	 2022-05-21 17:51:31
답변하는 윤석열 대통령 	 2022-05-21 17:51:31
한미 정상회담 관련 브리핑하는 김성한 안보실장 	 2022-05-21 17:51:01
한미 정상회담 관련 브리핑하는 김성한 안보실장 	 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 	 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 	 2022-05-21 17:51:01
한미 정상회담 관련 브리핑하는 김성한 안보실장 	 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 	 2022-05-21 17:51:01
변성완 부산시장 후보 지지 호소 	 2022-05-21 17:37:06
공동기자회견하는 한미 정상 	 2022-05-21 17:36:55
공동기자회견하는 한미 정상 	 2022-05-21 17:36:55
두 정상 발언 경청하는 추경호 경제부총리 	 2022-05-21 17:21:40
공동기자회견하는 한미 정상 	 2022-05-21 17:15:43
한미 정상 공동기자회견 	 2022-05-21 17:15:43
공동기자회견하는 한미 정상 	 2022-05-21 17:15:43
공동기자회견하는 한미 정상 	 2022-05-21 17:15:43
한미 정상 공동기자회견 	 2022-05-21 17:15:27
공동기자회견하는 한미 정상 	 2022-05-21 17:15:27
한미 정상 공동기자회견 	 2022-05-21 17:15:27
한미 정상 공동기자회견 	 2022-05-21 17:15:27
공동기자회견 참석하는 한미 정상 	 202

************** 2022-05-21 20:00:00 ~ 2022-05-21 20:59:59 군집화 결과********************
지지 호소하는 무소속 민광준 증평군수 후보 선거운동원들 	 2022-05-21 19:32:20
송기윤 증평군수 후보 지원유세 나선 독고영재 	 2022-05-21 19:32:20
합동 선거유세하는 민주당 증평지역 후보자들 	 2022-05-21 19:27:19
국민의힘 증평지역 후보 합동유세 	 2022-05-21 19:27:19
한미 정상 만찬사 하는 윤석열 대통령 	 2022-05-21 20:50:12
우리모두 투표 참여해요! 	 2022-05-21 20:44:20
우리모두 투표해요! 	 2022-05-21 20:44:20
투표로 밝히는 온 동네 	 2022-05-21 20:44:20
6.1지방선거 투표 참여합시다 	 2022-05-21 20:44:20
우리모두 투표해요! 	 2022-05-21 20:44:20
6.1지방선거 투표 참여합시다 	 2022-05-21 20:44:20
'우리모두 투표해요' 	 2022-05-21 20:44:20
한미 정상 만찬 	 2022-05-21 20:41:39
한미 정상 만찬 답사하는 조 바이든 미국 대통령 	 2022-05-21 20:41:39
한미 정상 만찬 답사하는 조 바이든 미국 대통령 	 2022-05-21 20:41:39
한미 정상 만찬 답사하는 조 바이든 미국 대통령 	 2022-05-21 20:41:39
윤석열 대통령-조 바이든 대통령, 한미 만찬 참석 	 2022-05-21 20:33:08
윤석열 대통령-조 바이든 대통령, 한미 만찬 참석 	 2022-05-21 20:32:25
윤석열 대통령-조 바이든 대통령, 한미 만찬 참석 	 2022-05-21 20:31:19
김민영 정읍시장 후보 유세  "선거율동은 이런 것" 	 2022-05-21 20:21:46
유권자에게 악수 청하는 무소속 김민영 정읍시장 후보 	 2022-05-21 20:17:33
지지호소하는 김

In [264]:
# 시작일,종료일 설정
start = "2022-05-21 00:00:00"
last = "2022-05-21 23:00:00"

# 시작일, 종료일 datetime 으로 변환
start_date = datetime.strptime(start, "%Y-%m-%d %H:%M:%S")
last_date = datetime.strptime(last, "%Y-%m-%d %H:%M:%S")

merge_dict = {}
current_dict = {}

# 종료일 까지 반복
while start_date <= last_date:
    
    end_date = start_date + timedelta(hours=1) + timedelta(seconds=-1)
    
    s_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    e_date = end_date.strftime("%Y-%m-%d %H:%M:%S")
    print(s_date, e_date)
    
    sub_df = sub_df_return(article_df, s_date, e_date)
    sub_vector = sub_vector_return(sub_df)
    article_dict = dict(cluster_return(sub_vector, sub_df))
    if len(merge_dict) == 0:
        merge_dict = article_dict.copy()
    else:
        current_dict = article_dict.copy()
        merge_dict = merge(merge_dict, current_dict).copy()
    
    print("**************",s_date,"~",e_date,"군집화 결과********************")
    # 클러스터의 중요도 판단
    weight = 0.63
    item_list = []
    for key, value in merge_dict.items():
        cluster = value
        cluster_size = len(cluster)
        cluster_score = 0
        for article in cluster:
            article_time = article['datetime']
            article_time = datetime.strptime(article_time, "%Y-%m-%d %H:%M:%S")
            if start_date <= article_time and article_time <= end_date:
                cluster_score += 1
        cluster_score = weight * cluster_size + (1-weight) * cluster_score
        item_list.append([key, cluster_score])
            
    item_list = sorted(item_list, key=lambda x : x[1], reverse=True)
    for item in item_list[:5]:
        cluster_num = item[0]
        cluster = merge_dict[cluster_num]
        for article in cluster:
            print(article['title'],"\t",article['datetime'])
        print("\n")
        
    # 시간이 오래된 값은 제거 필요
    remove_cluster_list = []
    for cluster_num in merge_dict:
        cluster = merge_dict[cluster_num]
        remove_list = []
        # 시간 차이를 구한뒤, 리스트의 index를 저장
        for i, article in enumerate(cluster):
            article_time = article['datetime']
            article_time = datetime.strptime(article_time, "%Y-%m-%d %H:%M:%S")
            
            diff = article_time - start_date
            if diff.seconds > 14400: # 4시간 이상 차이날 경우 제거 필요
                remove_list.append(i)
                
        # 지워야할 index를 pop 해준다
        remove_list.reverse() # 뒤에서 부터 지워야 하기 때문에 reverse 함수
        for i in remove_list:
            cluster.pop(i)
            
        # 지우고 보니 cluster에 남아있는게 없다면 해당 클러스터 제거
        if len(cluster) == 0:
            remove_cluster_list.append(cluster_num)
            
    # 지우고 보니 cluster에 남아있는게 없다면 해당 클러스터 제거
    for cluster_num in remove_cluster_list:
        del merge_dict[cluster_num]
        
    start_date += timedelta(hours=1)

2022-05-21 00:00:00 2022-05-21 00:59:59
************** 2022-05-21 00:00:00 ~ 2022-05-21 00:59:59 군집화 결과********************
[주간政談<하>] 달라진 5·18 기념식 풍경...민주당보다 돋보인 국민의힘 	 2022-05-21 00:02:08
[주간政談<상>] 윤석열-바이든-이재용 '3샷', 글로벌 홍보 효과 '톡톡' 	 2022-05-21 00:02:04
오늘 바이든 환영만찬에 여야 대표 나란히 참석 	 2022-05-21 00:01:10


거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합) 	 2022-05-21 00:24:12
이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합) 	 2022-05-21 00:11:04


"신천지, 부정적 이미지 세탁과 교세 확장 위해 정치권 접근" 	 2022-05-21 00:27:01


의사 출신 민주 신현영 의원, KTX 응급 환자 발생하자 달려갔다 	 2022-05-21 00:10:01


한덕수 인준 '파국 피했다'…野, 지지율 추락 위기 느꼈나 	 2022-05-21 00:04:01


2022-05-21 01:00:00 2022-05-21 01:59:59
************** 2022-05-21 01:00:00 ~ 2022-05-21 01:59:59 군집화 결과********************
거리 유세하던 이재명 향해 철제그릇 던진 60대 체포(종합) 	 2022-05-21 00:24:12
이재명, 인천 거리유세 중 취객이 던진 철제통에 맞아(종합) 	 2022-05-21 00:11:04
이재명, 유세 중 철제그릇에 머리 맞아... 가해자 체포 	 2022-05-21 01:26:01


尹-바이든, 최첨단 웨이퍼 서명으로 '기술동맹' 인증[바이든 방한] 	 2022-05-21 01:48:02
한국인 폭행 혐의 바이든 경호원 2명 美 송환…"마약복용 조사" 	 2022-05-21 01:4

************** 2022-05-21 11:00:00 ~ 2022-05-21 11:59:59 군집화 결과********************
尹 대통령, 한덕수 국무총리 임명… 지명 48일만 	 2022-05-21 10:56:02
尹 대통령, 한덕수 국무총리 임명 	 2022-05-21 10:44:01
윤석열 대통령, 한덕수 국무총리 임명..."같이 열심히 일하자" 	 2022-05-21 10:39:01
尹대통령, 한덕수 국무총리 임명... 1기 내각 본격 출범 	 2022-05-21 10:34:01
윤 대통령, 한덕수 신임 국무총리에 임명장 수여[경향포토] 	 2022-05-21 10:32:02
윤 대통령, 한덕수 국무총리 임명… 후보자 지목 48일만(상보) 	 2022-05-21 10:23:04
尹대통령, 한덕수 국무총리 임명…지명 48일만 	 2022-05-21 10:20:10
尹대통령, 한덕수 총리 임명 재가…지명 후 48일만 	 2022-05-21 10:17:07
尹대통령, 한덕수 국무총리 임명…지명 후 48일 만 	 2022-05-21 10:12:22
[속보] 尹 대통령, 한덕수 국무총리 임명 재가 	 2022-05-21 10:10:11
[속보] 윤 대통령, 한덕수 국무총리 임명 	 2022-05-21 10:10:01
[속보] 尹 대통령, 한덕수 국무총리 임명 재가 	 2022-05-21 10:05:52
[속보] 윤석열 대통령, 한덕수 국무총리 임명 재가 	 2022-05-21 10:05:02
윤 대통령, 한덕수 총리 임명…“열심히 일해달라” 	 2022-05-21 10:04:50
윤 대통령, 한덕수 국무총리 임명안 재가…임명장 수여 	 2022-05-21 10:04:49
[속보] 尹대통령, 한덕수 국무총리 임명 	 2022-05-21 10:04:13
尹 대통령, 한덕수 국무총리 임명 재가 	 2022-05-21 10:04:02
[속보] 尹대통령, 한덕수 국무총리 임명 	 2022-05-21 10:03:41
尹대통령, 한덕수 국무총리 임명…

************** 2022-05-21 14:00:00 ~ 2022-05-21 14:59:59 군집화 결과********************
[포토] 용산 대통령실 청사 도착한 바이든 미 대통령 	 2022-05-21 13:50:01
바이든, 용산 대통령실 도착…윤 대통령과 정상회담 돌입 	 2022-05-21 13:49:04
[속보] 바이든, 서울 용산 대통령실 도착…尹 대통령, 정문 밖에서 영접 	 2022-05-21 13:45:01
윤석열 대통령, 용산 청사 정문서 바이든 영접…정상회담 후 공동기자회견 	 2022-05-21 13:40:01
바이든 미국 대통령 영접하는 윤석열 대통령 	 2022-05-21 13:31:11
바이든 미국 대통령 영접하는 윤석열 대통령 	 2022-05-21 13:30:10
[속보] 바이든, 용산 대통령실 청사 도착...尹대통령, 정문 입구서 영접 	 2022-05-21 13:26:01
[속보] 바이든, 용산 대통령실 도착…尹, 정문서 직접 영접 	 2022-05-21 13:26:01
[영상] 바이든, 용산 대통령실 도착…윤 대통령, 정문서 직접 영접 	 2022-05-21 13:25:37
[속보] 바이든, 용산 대통령실 청사 도착 	 2022-05-21 13:25:19
[속보] 바이든, 용산 대통령실 청사 도착 	 2022-05-21 13:25:04
美 바이든 대통령 방한 둘째 날 현충원 참배…곧 정상회담 	 2022-05-21 13:13:43
尹, 용산청사 정문서 바이든 직접 영접…정상회담 시작 	 2022-05-21 13:12:01
尹·바이든, 90분 정상회담 시작…소인수→5분간 환담→확대 	 2022-05-21 14:48:57
용산 청사 온 바이든, 방명록에 "환대와 동맹에 감사"[바이든 방한] 	 2022-05-21 14:45:02
尹대통령, 용산청사 ‘레드카펫’서 바이든 맞이…정상회담 돌입(종합) 	 2022-05-21 14:42:01
확대정상회담에 한미 주요 인사 22명 참석...尹, 바이든과 공동선언문 

************** 2022-05-21 16:00:00 ~ 2022-05-21 16:59:59 군집화 결과********************
회담 참석자와 인사하는 윤석열 대통령 	 2022-05-21 15:58:19
[속보]바이든 "한미동맹, 北 위협 억제에 중요..오늘날 세계 안전유지에 필수" 	 2022-05-21 15:58:03
[속보] 尹대통령 "한미, 반도체·배터리 등 전략산업 분야 협력" 	 2022-05-21 15:56:13
[속보] 尹대통령 "한미동맹, 경제안보시대 맞춰 발전·진화해야" 	 2022-05-21 15:56:13
한미 정상회담 	 2022-05-21 15:55:11
한미 정상회담 	 2022-05-21 15:55:10
한미 정상회담 	 2022-05-21 15:55:09
[속보] 바이든 “한미동맹, 세계의 안전 유지하는 데 필수적” 	 2022-05-21 15:54:01
[속보영상] 용산 대통령실 ‘첫 외빈’ 바이든 “환대와 동맹에 감사” 	 2022-05-21 15:53:15
[속보] 한미 확대정상회담 종료…곧 기자회견 	 2022-05-21 15:47:10
회담 참석자와 인사하는 윤석열 대통령 	 2022-05-21 15:41:13
회담 참석자와 인사하는 윤석열 대통령 	 2022-05-21 15:41:12
대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:40:20
대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:40:19
발언하는 윤석열 대통령 	 2022-05-21 15:39:12
발언하는 바이든 대통령 	 2022-05-21 15:38:15
발언하는 바이든 대통령 	 2022-05-21 15:38:13
용산 대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:22:13
용산 대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:22:13
용산 대통령실 청사에서 열린 한미 정상회담 	 2022-05-21 15:22:12
용산 대통령실 청사에서 열린 한미 

박형준 후보, 시민들과 만남 통해 지지 호소 	 2022-05-21 17:56:52
공동기자회견 마친 한미 정상 	 2022-05-21 17:51:31
공동기자회견 마친 한미 정상 	 2022-05-21 17:51:31
답변하는 윤석열 대통령 	 2022-05-21 17:51:31
한미 정상회담 관련 브리핑하는 김성한 안보실장 	 2022-05-21 17:51:01
한미 정상회담 관련 브리핑하는 김성한 안보실장 	 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 	 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 	 2022-05-21 17:51:01
한미 정상회담 관련 브리핑하는 김성한 안보실장 	 2022-05-21 17:51:01
김성한 안보실장 '한미 정상회담 관련 브리핑' 	 2022-05-21 17:51:01
변성완 부산시장 후보 지지 호소 	 2022-05-21 17:37:06
공동기자회견하는 한미 정상 	 2022-05-21 17:36:55
공동기자회견하는 한미 정상 	 2022-05-21 17:36:55
두 정상 발언 경청하는 추경호 경제부총리 	 2022-05-21 17:21:40
공동기자회견하는 한미 정상 	 2022-05-21 17:15:43
한미 정상 공동기자회견 	 2022-05-21 17:15:43
공동기자회견하는 한미 정상 	 2022-05-21 17:15:43
공동기자회견하는 한미 정상 	 2022-05-21 17:15:43
한미 정상 공동기자회견 	 2022-05-21 17:15:27
공동기자회견하는 한미 정상 	 2022-05-21 17:15:27
한미 정상 공동기자회견 	 2022-05-21 17:15:27
한미 정상 공동기자회견 	 2022-05-21 17:15:27
공동기자회견 참석하는 한미 정상 	 2022-05-21 17:12:01
공동기자회견 참석하는 한미 정상 	 2022-05-21 17:12:01
공동기자회견 참석하는 한미 정상 	 2022-05

************** 2022-05-21 20:00:00 ~ 2022-05-21 20:59:59 군집화 결과********************
지지 호소하는 무소속 민광준 증평군수 후보 선거운동원들 	 2022-05-21 19:32:20
송기윤 증평군수 후보 지원유세 나선 독고영재 	 2022-05-21 19:32:20
합동 선거유세하는 민주당 증평지역 후보자들 	 2022-05-21 19:27:19
국민의힘 증평지역 후보 합동유세 	 2022-05-21 19:27:19
한미 정상 만찬사 하는 윤석열 대통령 	 2022-05-21 20:50:12
우리모두 투표 참여해요! 	 2022-05-21 20:44:20
우리모두 투표해요! 	 2022-05-21 20:44:20
투표로 밝히는 온 동네 	 2022-05-21 20:44:20
6.1지방선거 투표 참여합시다 	 2022-05-21 20:44:20
우리모두 투표해요! 	 2022-05-21 20:44:20
6.1지방선거 투표 참여합시다 	 2022-05-21 20:44:20
'우리모두 투표해요' 	 2022-05-21 20:44:20
한미 정상 만찬 	 2022-05-21 20:41:39
한미 정상 만찬 답사하는 조 바이든 미국 대통령 	 2022-05-21 20:41:39
한미 정상 만찬 답사하는 조 바이든 미국 대통령 	 2022-05-21 20:41:39
한미 정상 만찬 답사하는 조 바이든 미국 대통령 	 2022-05-21 20:41:39
윤석열 대통령-조 바이든 대통령, 한미 만찬 참석 	 2022-05-21 20:33:08
윤석열 대통령-조 바이든 대통령, 한미 만찬 참석 	 2022-05-21 20:32:25
윤석열 대통령-조 바이든 대통령, 한미 만찬 참석 	 2022-05-21 20:31:19
김민영 정읍시장 후보 유세  "선거율동은 이런 것" 	 2022-05-21 20:21:46
유권자에게 악수 청하는 무소속 김민영 정읍시장 후보 	 2022-05-21 20:17:33
지지호소하는 김